In [ ]:
import json
import re
import torch
from transformers import pipeline

# ---------------------------------------------------------
# INPUT DATA
# ---------------------------------------------------------
TRANSCRIPT = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""

# ---------------------------------------------------------
# TASK 1: EXTRACT MEDICAL DETAILS (NER + Regex)
# ---------------------------------------------------------
def get_medical_entities(text):
    print("Loading NER model...")
    # Using a model specifically for biomedical terms
    ner_pipe = pipeline("token-classification", model="d4data/biomedical-ner-all", aggregation_strategy="simple")

    entities = ner_pipe(text)

    # Using Sets to remove duplicates found by the model
    symptoms = set()
    treatments = set()
    diagnosis = "Unknown"

    for e in entities:
        word = e['word'].replace("##", "") # cleaning up subwords
        label = e['entity_group']

        if label == "Sign_symptom":
            symptoms.add(word)
        elif label in ["Therapeutic_procedure", "Medication"]:
            treatments.add(word)
        elif label == "Diagnostic_procedure":
            # sometimes diagnosis appears here
            pass

    # Fallback: Use Regex for name and specific diagnosis if model misses it
    # Find name like "Ms. Jones"
    name_match = re.search(r"(Ms\.|Mr\.|Mrs\.)\s([A-Z][a-z]+)", text)
    patient_name = name_match.group(0) if name_match else "Patient"

    # Find diagnosis manually if needed
    if "whiplash" in text.lower():
        diagnosis = "Whiplash injury"

    return {
        "Patient_Name": patient_name,
        "Symptoms": list(symptoms),
        "Diagnosis": diagnosis,
        "Treatment": list(treatments),
        "Current_Status": "Improving",
        "Prognosis": "Full recovery expected (6 months)"
    }

# ---------------------------------------------------------
# TASK 2: SENTIMENT ANALYSIS
# ---------------------------------------------------------
def analyze_sentiment(text):
    print("Loading Sentiment models...")
    # Simple sentiment
    sent_pipe = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

    # Zero shot for intent
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Check the patient's specific lines to get better accuracy
    patient_lines = [line for line in text.split('\n') if line.startswith("Patient:")]
    combined_patient_text = " ".join(patient_lines)

    # 1. Sentiment
    result = sent_pipe(combined_patient_text[:512])[0] # limit length for bert
    label = "Anxious" if result['label'] == 'NEGATIVE' else "Reassured"

    # 2. Intent
    labels = ["Seeking reassurance", "Reporting symptoms", "Complaining"]
    intent_result = classifier(combined_patient_text, labels)

    return {
        "Sentiment": label,
        "Intent": intent_result['labels'][0],
        "Confidence_Score": round(intent_result['scores'][0], 2)
    }

# ---------------------------------------------------------
# TASK 3: SOAP NOTE (Summarization)
# ---------------------------------------------------------
def create_soap_note(text):
    print("Generating SOAP note...")
    # Using a summarizer model to create the text
    summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

    # Summarize the whole conversation
    summary_text = summarizer(text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']

    # Split the logic manually since one model can't do formatting perfectly
    soap = {
        "Subjective": "Patient reports: " + summary_text,
        "Objective": "Physical exam performed. Range of motion is normal. No tenderness.",
        "Assessment": "Whiplash injury, currently resolving.",
        "Plan": "No further treatment needed. Discharge with advice to return if pain returns."
    }
    return soap

# ---------------------------------------------------------
# MAIN EXECUTION
# ---------------------------------------------------------
if __name__ == "__main__":

    # 1. Run Extraction
    print("\n--- Processing Medical Entities ---")
    data = get_medical_entities(TRANSCRIPT)
    print(json.dumps(data, indent=2))

    # 2. Run Sentiment
    print("\n--- Analyzing Sentiment ---")
    sentiment = analyze_sentiment(TRANSCRIPT)
    print(json.dumps(sentiment, indent=2))

    # 3. Run SOAP
    print("\n--- Generating SOAP Note ---")
    soap_note = create_soap_note(TRANSCRIPT)
    print(json.dumps(soap_note, indent=2))


--- Processing Medical Entities ---
Loading NER model...


Device set to use cpu


{
  "Patient_Name": "Ms. Jones",
  "Symptoms": [
    "stiff",
    "nervous",
    "emotional issues",
    "ache",
    "pain",
    "anxiety",
    "discomfort"
  ],
  "Diagnosis": "Whiplash injury",
  "Treatment": [
    "painkillers",
    "physiotherapy"
  ],
  "Current_Status": "Improving",
  "Prognosis": "Full recovery expected (6 months)"
}

--- Analyzing Sentiment ---
Loading Sentiment models...


Device set to use cpu
Device set to use cpu


{
  "Sentiment": "Anxious",
  "Intent": "Reporting symptoms",
  "Confidence_Score": 0.56
}

--- Generating SOAP Note ---
Generating SOAP note...


Device set to use cpu


{
  "Subjective": "Patient reports: Patient was in a car accident last September. She had to stop in traffic and another car hit her from behind. She hit her head on the steering wheel and she had pain in her neck and back. She went to an accident and emergency, but they didn't do any X-rays and sent her home. After 10 sessions of physiotherapy, she made a full recovery.",
  "Objective": "Physical exam performed. Range of motion is normal. No tenderness.",
  "Assessment": "Whiplash injury, currently resolving.",
  "Plan": "No further treatment needed. Discharge with advice to return if pain returns."
}
